In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
df = pd.read_csv('online_payment.csv')
del df['nameOrig']
del df['nameDest']

In [3]:
X = df.drop(['isFraud'],axis=1)
y = df['isFraud']

In [4]:
le = LabelEncoder()
var = X.select_dtypes(include='object').columns
for i in var:
    X[i] = le.fit_transform(X[i])

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            int32  
 2   amount          float64
 3   oldbalanceOrg   float64
 4   newbalanceOrig  float64
 5   oldbalanceDest  float64
 6   newbalanceDest  float64
 7   isFlaggedFraud  int64  
dtypes: float64(5), int32(1), int64(2)
memory usage: 364.1 MB


In [6]:
model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0,learning_rate=0.5,max_depth=6,sampling_method='gradient_based',max_bin=768,min_child_weight=1,gamma=0.2)

In [7]:
X_train , X_test , y_train , y_test = train_test_split(X,y, test_size=0.30 , random_state=672,stratify=y)

In [8]:
X_test.tail()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
689599,36,1,12660.25,106731.00,94070.75,13600.49,26260.74,0
1730708,161,3,9264.67,0.00,0.00,0.00,0.00,0
4199299,305,4,3779593.79,11583.00,0.00,758843.02,4538436.81,0
24628,8,3,151.79,531396.45,531244.66,0.00,0.00,0
6098363,522,3,60.32,0.00,0.00,0.00,0.00,0


In [16]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.5, max_bin=768, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [17]:
y_pred = model.predict(X_test)

In [18]:
X_test.shape

(1908786, 8)

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906322
           1       0.97      0.89      0.93      2464

    accuracy                           1.00   1908786
   macro avg       0.98      0.94      0.96   1908786
weighted avg       1.00      1.00      1.00   1908786



In [20]:
print('  confusion matrix  ')
print('')
print(confusion_matrix(y_test, y_pred))

  confusion matrix  

[[1906252      70]
 [    279    2185]]


In [21]:
print("Train Accuracy: ",model.score(X_train,y_train))
print("Test Accuracy: ",model.score(X_test,y_test))

Train Accuracy:  0.9999250084309383
Test Accuracy:  0.999817161274234


In [22]:
X_test.tail()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
689599,36,1,12660.25,106731.00,94070.75,13600.49,26260.74,0
1730708,161,3,9264.67,0.00,0.00,0.00,0.00,0
4199299,305,4,3779593.79,11583.00,0.00,758843.02,4538436.81,0
24628,8,3,151.79,531396.45,531244.66,0.00,0.00,0
6098363,522,3,60.32,0.00,0.00,0.00,0.00,0
